In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 12.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [19]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-07-06 17:29:36,619] A new study created in memory with name: no-name-31ad2ec9-21c5-4aa9-817c-8c3253fea0aa
[I 2025-07-06 17:29:40,225] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 136, 'max_depth': 12}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-06 17:29:41,773] Trial 1 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 148, 'max_depth': 15}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-06 17:29:42,897] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 137, 'max_depth': 16}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-06 17:29:43,950] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 136, 'max_depth': 17}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-07-06 17:29:44,495] Trial 4 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 61, 'max_depth': 17}. Best is trial 4 with value: 0.77653

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 20}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


***Random Search***

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-07-06 17:22:18,379] A new study created in memory with name: no-name-76803fe1-dbea-4ac7-8c92-009954a6193c
[I 2025-07-06 17:22:20,158] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 140, 'max_depth': 13}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-07-06 17:22:21,144] Trial 1 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 131, 'max_depth': 10}. Best is trial 1 with value: 0.7690875232774674.
[I 2025-07-06 17:22:22,471] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 185, 'max_depth': 5}. Best is trial 1 with value: 0.7690875232774674.
[I 2025-07-06 17:22:22,973] Trial 3 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 68, 'max_depth': 12}. Best is trial 1 with value: 0.7690875232774674.
[I 2025-07-06 17:22:24,486] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 199, 'max_depth': 8}. Best is trial 1 with value: 0.7690875

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 127, 'max_depth': 16}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


**Grid Search**

In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-07-06 17:25:06,215] A new study created in memory with name: no-name-0fa86d8e-0b63-4dd0-960e-749831bd802c
[I 2025-07-06 17:25:07,545] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-06 17:25:08,924] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-06 17:25:09,323] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-06 17:25:10,062] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-06 17:25:10,804] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


**Visualization**

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [20]:
# 1. Optimization History
plot_optimization_history(study).show()

In [21]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [22]:
# 3. Slice Plot
plot_slice(study).show()

In [23]:
# 4. Contour Plot
plot_contour(study).show()

In [24]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [25]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [26]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [27]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-07-06 17:36:40,511] A new study created in memory with name: no-name-051a3796-11dd-4574-a9e8-365baf89fa48
[I 2025-07-06 17:36:42,581] Trial 0 finished with value: 0.7709497206703911 and parameters: {'classifier': 'RandomForest', 'n_estimators': 200, 'max_depth': 16, 'min_samples_split': 10, 'min_samples_leaf': 7, 'bootstrap': False}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-07-06 17:36:46,235] Trial 1 finished with value: 0.7746741154562384 and parameters: {'classifier': 'RandomForest', 'n_estimators': 214, 'max_depth': 20, 'min_samples_split': 9, 'min_samples_leaf': 8, 'bootstrap': False}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-06 17:36:47,474] Trial 2 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 115, 'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 8, 'bootstrap': True}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-06 17:36:47,522] Trial 3 finished with value:

In [28]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.10715737724973357, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7877094972067038


In [29]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.770950,2025-07-06 17:36:40.514262,2025-07-06 17:36:42.581485,0 days 00:00:02.067223,NaN,False,RandomForest,NaN,NaN,NaN,16.0,7.0,10.0,200.0,COMPLETE
1,1,0.774674,2025-07-06 17:36:42.585375,2025-07-06 17:36:46.234898,0 days 00:00:03.649523,NaN,False,RandomForest,NaN,NaN,NaN,20.0,8.0,9.0,214.0,COMPLETE
2,2,0.759777,2025-07-06 17:36:46.238708,2025-07-06 17:36:47.474495,0 days 00:00:01.235787,NaN,True,RandomForest,NaN,NaN,NaN,9.0,8.0,2.0,115.0,COMPLETE
3,3,0.739292,2025-07-06 17:36:47.475708,2025-07-06 17:36:47.522149,0 days 00:00:00.046441,1.453753,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.765363,2025-07-06 17:36:47.524157,2025-07-06 17:36:48.987406,0 days 00:00:01.463249,NaN,False,RandomForest,NaN,NaN,NaN,16.0,3.0,5.0,225.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-07-06 17:37:39.825746,2025-07-06 17:37:39.894335,0 days 00:00:00.068589,0.264990,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.757914,2025-07-06 17:37:39.899794,2025-07-06 17:37:41.654580,0 days 00:00:01.754786,NaN,True,RandomForest,NaN,NaN,NaN,20.0,2.0,8.0,153.0,COMPLETE
97,97,0.752328,2025-07-06 17:37:41.655800,2025-07-06 17:37:41.708891,0 days 00:00:00.053091,0.832426,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.741155,2025-07-06 17:37:41.710700,2025-07-06 17:37:46.193577,0 days 00:00:04.482877,NaN,NaN,GradientBoosting,NaN,NaN,0.023844,10.0,4.0,3.0,205.0,COMPLETE


In [30]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,75
RandomForest,16
GradientBoosting,9


In [31]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.741775
RandomForest,0.765945
SVM,0.773731


In [32]:
# 1. Optimization History
plot_optimization_history(study).show()

In [33]:
# 3. Slice Plot
plot_slice(study).show()

In [34]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [ ]:
!pip install optuna-integration[xgboost]

In [37]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-07-06 17:46:02,606] A new study created in memory with name: no-name-654b4746-dd1f-469f-84dd-b38df5d82286


[0]	train-mlogloss:0.96471	eval-mlogloss:0.97973
[1]	train-mlogloss:0.70906	eval-mlogloss:0.70750
[2]	train-mlogloss:0.54394	eval-mlogloss:0.53247
[3]	train-mlogloss:0.45612	eval-mlogloss:0.43452
[4]	train-mlogloss:0.38235	eval-mlogloss:0.35527
[5]	train-mlogloss:0.31241	eval-mlogloss:0.27911
[6]	train-mlogloss:0.27512	eval-mlogloss:0.24139
[7]	train-mlogloss:0.24820	eval-mlogloss:0.21703
[8]	train-mlogloss:0.21481	eval-mlogloss:0.17822
[9]	train-mlogloss:0.19644	eval-mlogloss:0.15909
[10]	train-mlogloss:0.17893	eval-mlogloss:0.13836
[11]	train-mlogloss:0.17147	eval-mlogloss:0.12661
[12]	train-mlogloss:0.17001	eval-mlogloss:0.12494
[13]	train-mlogloss:0.16112	eval-mlogloss:0.11143
[14]	train-mlogloss:0.15936	eval-mlogloss:0.11334
[15]	train-mlogloss:0.15742	eval-mlogloss:0.11261
[16]	train-mlogloss:0.15426	eval-mlogloss:0.10859
[17]	train-mlogloss:0.15169	eval-mlogloss:0.10517
[18]	train-mlogloss:0.15029	eval-mlogloss:0.10540
[19]	train-mlogloss:0.14950	eval-mlogloss:0.10534
[20]	train

[I 2025-07-06 17:46:13,694] Trial 0 finished with value: 1.0 and parameters: {'lambda': 6.14635386720395e-05, 'alpha': 0.0011954948647539794, 'eta': 0.2635834741081382, 'gamma': 0.00020324350187080455, 'max_depth': 7, 'min_child_weight': 6, 'subsample': 0.9785313590790473, 'colsample_bytree': 0.43640362124248216}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.78277	eval-mlogloss:0.76080
[1]	train-mlogloss:0.62442	eval-mlogloss:0.58307
[2]	train-mlogloss:0.48584	eval-mlogloss:0.43082
[3]	train-mlogloss:0.37823	eval-mlogloss:0.31599
[4]	train-mlogloss:0.33017	eval-mlogloss:0.26588
[5]	train-mlogloss:0.28541	eval-mlogloss:0.21879
[6]	train-mlogloss:0.25249	eval-mlogloss:0.18744
[7]	train-mlogloss:0.25233	eval-mlogloss:0.18614
[8]	train-mlogloss:0.25089	eval-mlogloss:0.18550
[9]	train-mlogloss:0.24970	eval-mlogloss:0.18135
[10]	train-mlogloss:0.24648	eval-mlogloss:0.17707
[11]	train-mlogloss:0.24668	eval-mlogloss:0.17721
[12]	train-mlogloss:0.24661	eval-mlogloss:0.17717
[13]	train-mlogloss:0.24781	eval-mlogloss:0.18133
[14]	train-mlogloss:0.24698	eval-mlogloss:0.17942
[15]	train-mlogloss:0.24435	eval-mlogloss:0.17591
[16]	train-mlogloss:0.24251	eval-mlogloss:0.17433
[17]	train-mlogloss:0.24204	eval-mlogloss:0.17565
[18]	train-mlogloss:0.23177	eval-mlogloss:0.16319
[19]	train-mlogloss:0.23169	eval-mlogloss:0.16280
[20]	train

[I 2025-07-06 17:46:16,017] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.0008058365838044203, 'alpha': 5.436203327478982e-05, 'eta': 0.27980847834017875, 'gamma': 0.0056250657552911, 'max_depth': 7, 'min_child_weight': 6, 'subsample': 0.4610429906716529, 'colsample_bytree': 0.7966071220542591}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.79212	eval-mlogloss:0.77051
[1]	train-mlogloss:0.63561	eval-mlogloss:0.59480


[I 2025-07-06 17:46:44,858] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.87558	eval-mlogloss:0.88248
[1]	train-mlogloss:0.67500	eval-mlogloss:0.67266


[I 2025-07-06 17:46:45,265] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 6.14635386720395e-05, 'alpha': 0.0011954948647539794, 'eta': 0.2635834741081382, 'gamma': 0.00020324350187080455, 'max_depth': 7, 'min_child_weight': 6, 'subsample': 0.9785313590790473, 'colsample_bytree': 0.43640362124248216}
Best accuracy: 1.0


In [38]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()